In [1]:
from nltk import word_tokenize, pos_tag
from nltk.stem.wordnet import WordNetLemmatizer
import pandas as pd
import numpy as np
import time

In [2]:
# need to install all nltk dependences by doing:
# python -m nltk.downloader all

In [3]:
csv = pd.read_csv("/Users/garyhu/Desktop/Georgia Tech/Classes/CSE 6242/Project/podcast_sample.csv")

csv.head()

,id,genre,artist,duration,review_rating,review_count,text
0,275834665,Technology,Apple Inc.,2758.401752,4.088000,500,apple special events podcast keynote addresses...
1,880791662,Natural Sciences,"Real Ghost Stories Online | Paranormal, Supern...",2009.097500,3.890000,500,couple wakes mysterious orbs floating bed stra...
2,313139609,Wrestling,The Solomonster,2758.401752,4.867925,424,solomonster covered preview predictions fast l...
3,1245809419,Wrestling,Prime Time with Sean Mooney,5243.300341,4.784861,251,mooney vault relive legendary conversation bri...
4,1495463421,Running,Kirk Dewindt & Brakken Kraker,2758.401752,4.986486,222,month covers bit cross training nasal breathin...


In [4]:
# see full text
pd.set_option('display.max_colwidth', None)

In [5]:
df = csv
test_string = df.loc[1,'text']

In [6]:
def determine_tense_input(sentence):
    text = word_tokenize(sentence)
    tagged = pos_tag(text)
    print(tagged)
    tense = {}
    tense["future"] = len([word for word in tagged if word[1] == "MD"])
    tense["present"] = len([word for word in tagged if word[1] in ["VBP", "VBZ","VBG"]])
    tense["past"] = len([word for word in tagged if word[1] in ["VBD", "VBN"]]) 
    return(tense)

In [7]:
determine_tense_input(test_string)

[('couple', 'NN'), ('wakes', 'NNS'), ('mysterious', 'JJ'), ('orbs', 'IN'), ('floating', 'VBG'), ('bed', 'NN'), ('strange', 'JJ'), ('neighbour', 'NN'), ('unexplained', 'JJ'), ('entity', 'NN'), ('girl', 'NN'), ('wandering', 'VBG'), ('abandoned', 'VBN'), ('barn', 'IN'), ('haunting', 'VBG'), ('encounter', 'NN'), ('listener', 'NN'), ('feels', 'NNS'), ('odd', 'VBP'), ('sense', 'NN'), ('peace', 'NN'), ('fear', 'NN'), ('real', 'JJ'), ('ghost', 'NN'), ('story', 'NN'), ('supernatural', 'JJ'), ('event', 'NN'), ('report', 'NN'), ('write', 'VBP'), ('air', 'NN'), ('support', 'NN'), ('exp', 'VBD'), ('extra', 'JJ'), ('podcast', 'NN'), ('person', 'NN'), ('bonus', 'NN'), ('episode', 'NN'), ('week', 'NN'), ('support', 'NN'), ('exp', 'NN'), ('ghosts', 'NNS'), ('ghoststories', 'NNS'), ('halloween', 'VBP'), ('horror', 'NN'), ('paranormal', 'JJ'), ('supernatural', 'JJ'), ('haunting', 'NN'), ('haunted', 'VBN'), ('demonic', 'JJ'), ('hauntedhouse', 'NN'), ('cemetery', 'NN'), ('exp', 'NN'), ('ghoststory', 'NN'),

{'future': 3, 'present': 4902, 'past': 2582}

In [10]:
def change_tense(sentence):
    text = word_tokenize(sentence)
    tense = pos_tag(text) 
    
    verb_tags = ['VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ']
    
#     nltk list of verb tags and meanings    
#     VB verb, base form take
#     VBD verb, past tense took
#     VBG verb, gerund/present participle taking
#     VBN verb, past participle taken
#     VBP verb, sing. present, non-3d take
#     VBZ verb, 3rd person sing. present takes

    word_list = []
    
    for word in tense:
        if word[1] in verb_tags:
            w = word[0]
            base_form = WordNetLemmatizer().lemmatize(w, 'v')
            word_list.append(base_form)

        else:
            w = word[0]
            word_list.append(w)
    
    final = ' '.join(word_list)
    return final

In [11]:
start = time.time()
df2 = df
df2["new_tense"] = df2["text"].apply(lambda x: change_tense(str(x)))
end = time.time()
print(end-start)

32.974895000457764


In [19]:
from collections import defaultdict

def remove_dupe_count(sentence):
    list_of_words = sentence.split(' ')
    
    count_dict = defaultdict(int)
    
    for word in list_of_words:
        count_dict[word] += 1
    
    return len(count_dict)

def after_tense_count(sentence):
    sentence_2 = change_tense(sentence)
    list_of_words = sentence_2.split(' ')
    
    count_dict = defaultdict(int)
    
    for word in list_of_words:
        count_dict[word] += 1
    
    return len(count_dict)

df2['unique count'] = df2['text'].apply(lambda x: remove_dupe_count(x))
df2['count after tense'] = df2['text'].apply(lambda x: after_tense_count(x))

In [26]:
x = pd.Series(df2['unique count']).sum()
print(x)

147331


In [27]:
y = pd.Series(df2['count after tense']).sum()
print(y)

138207


In [28]:
x - y

9124